# CH16 Dynamic Programming

In [1]:
# Some important notes:
# DP is a general technique for solving optimization, search, and counting problems that can be decomposed into subproblems.
# what makes DP different is that the same subproblem may reoccur. Therefore, a key to making DP efficient is caching the results of intermediate computations
# The key to solving a DP problem efficiently is finding a way to break the problem into subproblems such that:
# - the original problem can be solved relatively easily once solutions to the subproblems are available, and
# - these subproblem solutions are cached.
# Top tips for DP:
# - Use DP whenever we have to make choices to arrive at the solution. Specifically, when you construct solution to the given instance from solutions to subinstances of smaller problems of the same kind.
# - Apart from optimization problems, DP is also applicable to counting and decision problems
# - Although conceptually DP involves recursion, often for efficiency the cache is built "bottom up", i.e., iteratively. 
# -- when DP is implemented recursively, the cache data structure is hash table or BST
# -- when DP is implemented iteratively, the cache data structure is one or multi-dimensional array
# - DP is based on combining solutions to subproblems to yield a solution to the original problem which might not work for all problems

### Fibonacci Series

In [9]:
# Fibonacci Series Time Complexity: O(N) Space Complexity: O(N)
def fibonacci(n, cache = {}):
    if n <= 1:
        return n
    elif n not in cache.keys():
        cache[n] = fibonacci(n-1) + fibonacci(n-2)
    return cache[n]

print(fibonacci(7))

# Time Complexity: O(N) Space Complexity: O(1)
def fibonacci_space_opt(n):
    if n <= 1:
        return n
    
    f_minus_2, f_minus_1 = 0, 1
    for _ in range(1, n):
        f = f_minus_2 + f_minus_1
        f_minus_2, f_minus_1 = f_minus_1, f
    return f_minus_1 # as f is stored in f_minus_1 in the last step

print(fibonacci_space_opt(7))

13
13


### Maximum sum over all subarrays

In [13]:
# Task: Find the maximum sum over all subarrays of a given array of integer. The array can have positive and negative integers.
# we need to find continous elements with maximum sum. 
# - If all the elements of the array are positive => max_sum = sum of all elements of the array
# - If all the elements of the array are negative => max_sum = max element of the array

# Brute Force: we can form all sub arrays and find each sum and then find max using two for loops. First, subarrays of size 1 then subarrays of size 2 etc.
# Total number of subarrays: ((n*(n+1))/2) and time to compute sum of each subarray is O(n)=>Time Complexity:O(n^3)
def max_sum_subarray_brute_force(a):
    ans = float('-inf')
    
    for sub_array_size in range(1,len(a) + 1):
        for start_index in range(0, len(a)):
            if (start_index + sub_array_size) > len(a): # subarray exceeds array bounds
                break
            sum_val = 0
            for i in range(start_index, start_index + sub_array_size):
                sum_val += a[i]
            ans = max(ans, sum_val)
    return ans
print(f'Brute Force:')
a = [904, 40, 523, 12, -335, -385, -124, 481, -31]
print(f'The max sum is {max_sum_subarray_brute_force(a)}')
a = [-2, -5, 6, -2, -3, 1, 5, -6]
print(f'The max sum is {max_sum_subarray_brute_force(a)}')

# Brute force algo can be improved by first computing S[k]=sigma(A[0,k]) for all k. Then sum for A[i,j] is then S[j] - S[i-1]. Time complexity can be reduced to O(n^2) in this method.
def max_sum_subarray_brute_force_opt(a):
    ans = float('-inf')
    sum_till_i = []
    sum_till_i[0] = a[0]
    for i in range(1, len(a)): # sub array sum till k is computed
        sum_till_i[i] += sum_till_i[i-1] + a[i]
    
    # Generating all sub arrays and finding their sum
    for sub_array_size in range(1, len(a) + 1):
        for start_index in range(0, len(a)):
            if start_index + sub_array_size > len(a):
                break
            end_index = start_index + sub_array_size - 1
            sum_val = sum_till_i[end_index] - sum_till_i[start_index-1]
            ans = max(ans, sum_val)
    return ans
print(f'\nBrute Force Optimized:')
a = [904, 40, 523, 12, -335, -385, -124, 481, -31]
print(f'The max sum is {max_sum_subarray_brute_force(a)}')
a = [-2, -5, 6, -2, -3, 1, 5, -6]
print(f'The max sum is {max_sum_subarray_brute_force(a)}')

# Divide and conquer: The approach is similar to quick sort. Split array into two halves => compute max sum in left and right=> max subarray sum is going to be max(l+r)
# Time Complexity: O(nlogn)
def max_sum_subarray_divide_and_conquer(a):
    def max_sum_subarray_divide_and_conquer_helper(a, start, end):
        if start == end:
            return a[start]
        
        m = (start + ((end - start)//2))
        sum_l = max_sum_subarray_divide_and_conquer_helper(a, 0, m)
        sum_r = max_sum_subarray_divide_and_conquer_helper(a, m+1, end)
        ans = max(ans, sum_l + sum_r)
        return ans
    
    return max_sum_subarray_divide_and_conquer_helper(a, 0, (len(a) - 1))

print(f'\nDivide and Conquer:')
a = [904, 40, 523, 12, -335, -385, -124, 481, -31]
print(f'The max sum is {max_sum_subarray_brute_force(a)}')
a = [-2, -5, 6, -2, -3, 1, 5, -6]
print(f'The max sum is {max_sum_subarray_brute_force(a)}')

# Kadane's Algorithm: The approach uses dynamic programming and its time complexity is O(n) and space complxity is O(1).
# Ref: https://www.baeldung.com/java-maximum-subarray
# Formula: maximumSubArraySum[i] = maximumSubArraySum[i-1] + arr[i] if arr[i] < maximumSubArraySum[i-1] + arr[i] else maximumSubArraySum[i] = arr[i]
def max_sum_subarray_DP(a):
    max_so_far = float('-inf')
    max_ending_here = float('-inf')
    for i in range(0, len(a)):
        if a[i] > max_ending_here + a[i]: # starting a new sub array
            start = i
            max_ending_here = a[i]
        else:
            max_ending_here = max_ending_here + a[i] # adding element to the earlier sub array
        
        if max_so_far < max_ending_here: # updating global maximum
            max_so_far = max_ending_here
            end = i
    print(f'Found maximum sum between {start} and {end} and the maximum sum is {max_so_far}')
    return
print(f'\nKadanes Algorithm DP:')
a = [904, 40, 523, 12, -335, -385, -124, 481, -31]
max_sum_subarray_DP(a)
a = [-2, -5, 6, -2, -3, 1, 5, -6]
max_sum_subarray_DP(a)

Brute Force:
The max sum is 1479
The max sum is 7

Brute Force Optimized:
The max sum is 1479
The max sum is 7

Divide and Conquer:
The max sum is 1479
The max sum is 7

Kadanes Algorithm DP:
Found maximum sum between 0 and 3 and the maximum sum is 1479
Found maximum sum between 2 and 6 and the maximum sum is 7


## 16.1 Count the numeber of score combinations

In [ ]:
# In an American football Bame, a play can lead to 2 points (safety), 3 points (field goal), or 7 points (touchdown, assuming the extra point). 
# Many different combinations of 2,3, and 7 point plays can make up a final score. For example, four combinations of plays yield a score of 12: o 6safeties (2x6=12), o 3 safeties and 2 field goals (2 x 3 + 3 x2 = 12), o 1 safety,l field goal and l touchdown (2 x 1+ 3 x 1 +7 x1= 12), and o 4fieldgoals(3x4=12).
# Write a program that takes a final score and scores for individual plays, and returns the number of combinations of plays that result in the final score.
